<a href="https://colab.research.google.com/github/OlanreleOladeji/Demand-forecast--XGBoost/blob/main/Products_quantity_forecasting_model_(XGB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data upload and preprocessing

In [279]:
import pandas as pd

In [280]:
import numpy as np

# forecast monthly births with xgboost
from numpy import asarray

In [281]:
from google.colab import files

In [282]:
uploaded = files.upload()

Saving 1pharma-data.csv to 1pharma-data (4).csv


In [283]:
df = pd.read_csv('1pharma-data.csv')

In [284]:
df.head()

,Distributor,Customer Name,City,Country,Latitude,Longitude,Channel,Sub-channel,Product_ID,Product Class,Sales,Price,Revenue,Month,Year,Name of Sales Rep,Manager,Sales Team,Unnamed: 18
0,Gottlieb-Cruickshank,"Zieme, Doyle and Kunze",Lublin,Poland,51.2333,22.5667,Hospital,Private,Topipizole,Mood Stabilizers,4.0,368,1472.0,January,2018,Mary Gerrard,Britanny Bold,Delta,NaN
1,Gottlieb-Cruickshank,Feest PLC,Świecie,Poland,53.4167,18.4333,Pharmacy,Retail,Choriotrisin,Antibiotics,7.0,591,4137.0,January,2018,Jessica Smith,Britanny Bold,Delta,NaN
2,Gottlieb-Cruickshank,Medhurst-Beer Pharmaceutical Limited,Rybnik,Poland,50.0833,18.5000,Pharmacy,Institution,Acantaine,Antibiotics,30.0,66,1980.0,January,2018,Steve Pepple,Tracy Banks,Bravo,NaN
3,Gottlieb-Cruickshank,Barton Ltd Pharma Plc,Czeladź,Poland,50.3333,19.0833,Hospital,Private,Lioletine Refliruvax,Analgesics,6.0,435,2610.0,January,2018,Mary Gerrard,Britanny Bold,Delta,NaN
4,Gottlieb-Cruickshank,Keeling LLC Pharmacy,Olsztyn,Poland,53.7800,20.4942,Pharmacy,Retail,Oxymotroban Fexoformin,Analgesics,20.0,458,9160.0,January,2018,Anne Wu,Britanny Bold,Delta,NaN


Check dataset structure

In [285]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254082 entries, 0 to 254081
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Distributor        254082 non-null  object 
 1   Customer Name      254082 non-null  object 
 2   City               254082 non-null  object 
 3   Country            254082 non-null  object 
 4   Latitude           254082 non-null  float64
 5   Longitude          254082 non-null  float64
 6   Channel            254082 non-null  object 
 7   Sub-channel        254082 non-null  object 
 8   Product_ID         254082 non-null  object 
 9   Product Class      254082 non-null  object 
 10  Sales              254082 non-null  float64
 11  Price              254082 non-null  int64  
 12  Revenue            254082 non-null  float64
 13  Month              254082 non-null  object 
 14  Year               254082 non-null  int64  
 15  Name of Sales Rep  254082 non-null  object 
 16  Ma

Data Exploration and Cleaning

Print the column headings

In [286]:
print(df.columns.tolist())


['Distributor', 'Customer Name', 'City', 'Country', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product_ID', 'Product Class', 'Sales', 'Price', 'Revenue', 'Month', 'Year', 'Name of Sales Rep', 'Manager', 'Sales Team', 'Unnamed: 18']


Dropping columns

In [287]:
df= df.drop(['Channel', 'Sub-channel', 'Product Class', 'Name of Sales Rep', 'Price','Manager', 'Sales Team', 'Customer Name', 'City', 'Country', 'Distributor', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product Class', 'Unnamed: 18'], axis=1)

In [232]:
df.head()

,Year,Month,Product_ID,Sales,Revenue
0,2017,April,Abatatriptan,1410.0,1046220.0
1,2017,April,Abilovir Aprotasol,1974.0,531006.0
2,2017,April,Abobozolid,1350.0,101250.0
3,2017,April,Abranatal Lysoprosate,3767.0,2565327.0
4,2017,April,Abtasol,594.0,447876.0


Sum the number of items in each month per year

In [288]:
monthly_sales = df.groupby(['Year', 'Month', 'Product_ID'])['Sales'].sum().reset_index()

In [289]:
Monthly_Revenue= df.groupby(['Year', 'Month', 'Product_ID'])['Revenue'].sum().reset_index()

In [290]:
merged_df = pd.merge(monthly_sales, Monthly_Revenue, on=['Year', 'Month', 'Product_ID'])

In [291]:
merged_df.head()

,Year,Month,Product_ID,Sales,Revenue
0,2017,April,Abatatriptan,1410.0,1046220.0
1,2017,April,Abilovir Aprotasol,1974.0,531006.0
2,2017,April,Abobozolid,1350.0,101250.0
3,2017,April,Abranatal Lysoprosate,3767.0,2565327.0
4,2017,April,Abtasol,594.0,447876.0


In [292]:
df= merged_df

In [293]:
df

,Year,Month,Product_ID,Sales,Revenue
0,2017,April,Abatatriptan,1410.0,1046220.0
1,2017,April,Abilovir Aprotasol,1974.0,531006.0
2,2017,April,Abobozolid,1350.0,101250.0
3,2017,April,Abranatal Lysoprosate,3767.0,2565327.0
4,2017,April,Abtasol,594.0,447876.0
...,...,...,...,...,...
11515,2020,September,Zithrotrol,3609.0,494433.0
11516,2020,September,Zonitonin,1300.0,673400.0
11517,2020,September,Zonixamine Immulinum,435.0,129630.0
11518,2020,September,Zynosine,790.0,58460.0


In [294]:
#print(df['Product_ID'].unique())

Feature Engineering

Split Dataset

In [295]:
from sklearn.model_selection import train_test_split
X = df[['Revenue']]
y = df[['Sales']]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)


In [296]:
X

,Revenue
0,1046220.0
1,531006.0
2,101250.0
3,2565327.0
4,447876.0
...,...
11515,494433.0
11516,673400.0
11517,129630.0
11518,58460.0


In [297]:
y

,Sales
0,1410.0
1,1974.0
2,1350.0
3,3767.0
4,594.0
...,...
11515,3609.0
11516,1300.0
11517,435.0
11518,790.0


Dataset Scaling using MinMaxScaler

In [298]:
from sklearn.preprocessing import MinMaxScaler

Scaling the train_X and test_X :

In [299]:
scaler_X = MinMaxScaler()

In [300]:
scaler_X.fit(X)

MinMaxScaler()

In [301]:
train_X_Scaled = scaler_X.transform(train_X)

In [302]:
train_X_Scaled = pd.DataFrame(train_X_Scaled, columns=['Revenue'])

In [237]:
train_X_Scaled.head(10)

,Revenue
0,0.049249
1,0.048355
2,0.070333
3,0.058062
4,0.049088
5,0.051306
6,0.069561
7,0.052523
8,0.047038
9,0.053201


In [303]:
test_X_Scaled = scaler_X.transform(test_X)

In [304]:
test_X_Scaled = pd.DataFrame(test_X_Scaled, columns=['Revenue'])

In [305]:
test_X_Scaled.head(10)

,Revenue
0,0.048843
1,0.055124
2,0.052252
3,0.081784
4,0.055866
5,0.051466
6,0.048486
7,0.072890
8,0.052004
9,0.068085


Scaling the train_y and test_y :

In [306]:
scaler_y = MinMaxScaler()

In [307]:
scaler_y.fit(y)

MinMaxScaler()

In [308]:
train_y_Scaled = scaler_y.transform(train_y)

In [309]:
train_y_Scaled = pd.DataFrame(train_y_Scaled, columns=['Sales'])

In [310]:
train_y_Scaled.head(10)

,Sales
0,0.061046
1,0.062750
2,0.070812
3,0.061553
4,0.059503
5,0.056642
6,0.076824
7,0.055990
8,0.053812
9,0.061569


In [311]:
test_y_Scaled = scaler_y.transform(test_y)

In [312]:
test_y_Scaled = pd.DataFrame(test_y_Scaled, columns=['Sales'])

In [313]:
test_y_Scaled.head(10)

,Sales
0,0.053989
1,0.058418
2,0.056537
3,0.093880
4,0.069936
5,0.057445
6,0.056199
7,0.091831
8,0.055291
9,0.070490


XGBoost Algorithm

In [314]:
from xgboost import XGBRegressor

In [315]:
model = XGBRegressor()

In [316]:
model.fit(train_X_Scaled, train_y_Scaled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Make predictions on the test dataset (test_X_Scaled):

In [317]:
y_predict = model.predict(test_X_Scaled)

In [318]:
print(predictions)

[0.0612699  0.06489413 0.06278524 ... 0.06202662 0.06340696 0.06214616]


Summarize progress

In [321]:
Progress_summary_df = pd.DataFrame({
        'Actual': test_y_Scaled['Sales'],
        'Predicted': predictions[:test_y_Scaled.shape[0]]
    })

print("Progress summary:")

display(Progress_summary_df.head(5))

Progress summary:


,Actual,Predicted
0,0.053989,0.061270
1,0.058418,0.064894
2,0.056537,0.062785
3,0.093880,0.089145
4,0.069936,0.065230


Evaluate model with performance

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

In [ ]:
mean_squared_error(test_y_Scaled, predictions)

0.00038125834544189274

In [ ]:
mean_absolute_error(test_y_Scaled, predictions)

0.008018013089895248

In [ ]:
 root_mean_squared_error(test_y_Scaled, predictions)

0.019525837153196335

In [ ]:
#pip install m5-wrmsse

In [ ]:
#from m5_wrmsse import wrmsse

In [ ]:
#y_predict = np.ones((30490,28))

In [ ]:
#wrmsse_score = wrmsse(y_predict)

In [ ]:
#print(wrmsse_score)

Hyperparemeter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    "learning_rate": [0.01, 0.1],
    "max_depth": [3, 7],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.1],
   }

In [ ]:
grid_search = GridSearchCV(estimator= XGBRegressor(), param_grid= param_grid, cv=5, verbose=1)

In [ ]:
grid_search.fit(train_X_Scaled, train_y_Scaled)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [3, 7],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700,
                                          800, 900, 1000],
                         'subsample': [0.8, 1.0]},
             verbose=1)

Best parameters

In [ ]:
grid_search.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 100,
 'subsample': 0.8}

Make predictions on the test dataset (test_X_Scaled) using grid search best estimator

In [ ]:
predictions_tuned = y_predict_tuned = grid_search.best_estimator_.predict(test_X_Scaled)

In [ ]:
print(predictions_tuned)

[0.06137752 0.06510728 0.06286702 ... 0.06199182 0.06261616 0.06199182]


Summarize progress after tunning

In [328]:
Progress_summary_df = pd.DataFrame({
        'Actual': test_y_Scaled['Sales'],
        'Predicted_tuned': predictions_tuned[:test_y_Scaled.shape[0]]
    })

print("Progress summary:")

display(Progress_summary_df.head(5))

Progress summary:


,Actual,Predicted_tuned
0,0.053989,0.061378
1,0.058418,0.065107
2,0.056537,0.062867
3,0.093880,0.089279
4,0.069936,0.065894


Evaluate model with performance after tunning

In [329]:
mean_squared_error(test_y_Scaled, predictions_tuned)

0.00038102251710370183

In [330]:
mean_absolute_error(test_y_Scaled, predictions_tuned)

0.007939578965306282

In [331]:
 root_mean_squared_error(test_y_Scaled, predictions_tuned)

0.019519798457622528

In [ ]:
#from m5_wrmsse import wrmsse

In [ ]:
#y_pred_tuned = np.ones((30490,28))

In [ ]:
#wrmsse_score = wrmsse(y_pred_tuned)
#print(wrmsse_score)